## R-CNN
- Regions with CNN features
- Rich feature hierarchies for accurate object detection and semantic segmentation tech report (paper)

![rcnn](img/rcnn.png)

![RCNN_Structure](img/RCNN_Structure.png)

### 모듈 1. Region Proposal
- (1) Object가 있을 만한 물체의 영역(Region)을 찾는 모듈
    - **기존의 Sliding window 방식의 비효율성 극복**
- (2) Selective Search를 이용한 region proposal output (2000개)
    - non-object-based segmentation
        - 객체와 주변간의 색감, 질감차이, 다른 물체에 애워쌓여있는지(Enclose) 여부 파악해서 다양한 전략으로 물체의 위치를 파악
    - Bottom-up 방식
        - small segmented areas들을 합쳐서 => 더 큰 segmentated areas를 만듬
        - Bounding box들을 Random하게 많이 생성을 하고 이들을 조금씩 merge 해나가면서 최종 물체를 박스로 판단

- (3) CNN을 위한 정사각형 이미지로 만들어 주기 위한 warp 작업
    - **왜 동일 Input 이어야 하는가?**
        - 사실 Convolution 연산은 input size가 고정일 필요는 없음 
        - **but. fc layer 에서는 input size (input node)가 고정이어야 함.**

![selective_search](img/selective_search.PNG)

### 모듈 2. CNN Feature Extraction
- Warping 된 image 를 CNN에 넣어줌 (224x224)
- AlexNet을 Detection class갯수에 맞게 변형한 구조를 이용 (거의 유사)
- 각각의 영역으로부터 고정된 크기의 Feature Vector를 만듬
    - ex) 어떤 박스의 영역이 결국에는 224x224로 변형되고 Feature Vector를 생성함. 그리고 그 영역을 SVM을 따라서 Class를 만듬.

### 3. Classification
- SVM Classification
    - 분류를 위한 선형 지도학습 모델 Support Vector Machine(SVM) 사용
    - **CNN으로 부터 추출된 각각의 Feature vector들의 점수를 class별로 매김**
    - **SVM class 결과로부터 객체인지 아닌지 Classifier 판별**

- BoundingBox(BB) Reg 조정
    - Selective search로 만든 Bounding box는 정확하지 않기 때문에 물체를 정확히 감싸도록 조정해주는 bounding box regression(선형회귀 모델) 이 존재
    - **localization error를 줄이기 위한 CNN Feature를 이용한 bounding box regression model 수정**
- **왜 Classification으로 Softmax대신 SVM 을 사용했는가?**
    - CNN Fine-tunning을 위한 학습 데이터가 시기상 많지 않아서 Softmax를 적용시키면 오히려 성능이 낮아짐
    
### 4. R-CNN Process
- (1) ImageNet Classification 데이터로 ConvNet을 pre-train 시켜 모델 M을 획득
- (2) 모델 M을 기반으로, Object detection 데이터로 ConvNet을 fine-tuning 시킨 모델 M' 획득
- (3) Object detection 데이터 각각의 이미지에 존재하는 모든 region proposal들에 대해 모델 M' 으로 Feature vector F를 추출하여 저장
- (4-1) 추출된 F를 기반으로 Classifier(SVM) 학습
- (4-2) 추출된 F를 기반으로 Linear bounding-box regressor 학습

### 단점
- 1. 작업이 오래걸린다
    - **Selective Search에서 뽑아낸 2000개의 영역 이미지들(Region proposal)에 대해서 모두 CNN 모델에 Classification을 하기 때문에 CNN연산*2000만큼의 시간이 걸려 수행시간이 매우 느리다.**

    - Training, Testing Time은 GPU K40 사용 기준 Frame 당 13초 / CPU : 53초
    
- 2. 복잡하다
    - CNN, SVM, Bounding Box Regression 총 3가지 모델이 multi-stage pipelines으로 한 번에 학습되지 않음
    - **각 region proposal에 대해 ConvNet Forward pass를 실행시 연산을 공유하지 않으므로 end-to-end로 학습 불가능**

    
- 3. Back Propagation이 안된다. (SVM, Bbox Reg -> CNN 불가)
    - **SVM, Bounding Box Regression에서 학습한 결과가 CNN을 업데이트 시키지 못함**
    


## 참고문헌
- https://ganghee-lee.tistory.com/35